# training data clean

In [63]:
import pandas as pd
import numpy as np
from numpy import savetxt
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
import sys
# np.set_printoptions(threshold=sys.maxsize)
import os
import matplotlib.pyplot as plt
from sklearn import datasets
import us as usStates

In [64]:
from datetime import datetime

def days(d1, d2):
    d1 = datetime.strptime(d1, "%m/%d/%Y")
    d2 = datetime.strptime(d2, "%m/%d/%Y")
    return abs((d2 - d1).days)+1

In [65]:
# from datetime import datetime

def weeks(d1, d2):
    d1 = datetime.strptime(d1, "%m/%d/%Y")
    d2 = datetime.strptime(d2, "%m/%d/%Y")
    return (abs((d2 - d1).days)+1)//7

In [66]:
def shuffle_in_unison(a, b):
    assert len(a) == len(b)
    shuffled_a = np.empty(a.shape, dtype=a.dtype)
    shuffled_b = np.empty(b.shape, dtype=b.dtype)
    permutation = np.random.permutation(len(a))
    for old_index, new_index in enumerate(permutation):
        shuffled_a[new_index] = a[old_index]
        shuffled_b[new_index] = b[old_index]
    return shuffled_a, shuffled_b
# x,y = shuffle_in_unison(x, y)

#### ground truth for flu

In [67]:
# weeks to drop
lst_weeks_drop = ['2022-06-18', '2022-06-25','2022-07-02','2022-07-09','2022-07-16', \
    '2022-10-22', '2022-10-29', '2022-11-05', '2022-11-12']
# states to drop 
states_to_drop = ['11']

In [68]:
dfg = pd.read_csv('/Users/xiangyikong/Desktop/week73/truth-Incident Hospitalizations.csv')

In [69]:
dfg['date'] = pd.to_datetime(dfg['date'])

In [70]:
dfg_date_filtered = dfg[(dfg['date'] >= '2022-02-26') & (dfg['date'] <= '2022-11-12')]
dfg_date_filtered = dfg_date_filtered[~dfg_date_filtered['date'].isin(lst_weeks_drop)]
dfg_date_filtered = dfg_date_filtered[~dfg_date_filtered['location'].isin(states_to_drop)]
# only get the states that are in the prediction
dfg_date_filtered_temp = dfg_date_filtered[dfg_date_filtered['location'].isin(list2_week4)]
# only get the dates that are in the prediction
dfg_date_filtered_temp = dfg_date_filtered_temp[dfg_date_filtered_temp['date'].isin(list1_week4)]
dfg_date_filtered_temp = dfg_date_filtered_temp.sort_values(by=['date', 'location'])

In [71]:
flu_groundtruth = np.resize(dfg_date_filtered_temp.groupby(['date','location']).sum().to_numpy().T, (16,51))

In [72]:
flu_groundtruth = flu_groundtruth.reshape(1,16,51)

In [73]:
dfg_date_filtered_temp['date'].unique()

array(['2022-02-26T00:00:00.000000000', '2022-03-05T00:00:00.000000000',
       '2022-03-12T00:00:00.000000000', '2022-03-19T00:00:00.000000000',
       '2022-03-26T00:00:00.000000000', '2022-04-02T00:00:00.000000000',
       '2022-04-09T00:00:00.000000000', '2022-04-16T00:00:00.000000000',
       '2022-04-23T00:00:00.000000000', '2022-04-30T00:00:00.000000000',
       '2022-05-07T00:00:00.000000000', '2022-05-14T00:00:00.000000000',
       '2022-05-21T00:00:00.000000000', '2022-05-28T00:00:00.000000000',
       '2022-06-04T00:00:00.000000000', '2022-06-11T00:00:00.000000000'],
      dtype='datetime64[ns]')

In [74]:
dfg_date_filtered_temp.groupby(['date','location']).sum()

value
date       location       
2022-02-26 01           14
           02            1
           04           72
           05           42
           06           34
...                    ...
2022-06-11 53           59
           54           15
           55           15
           56            6
           US         2224

[816 rows x 1 columns]

In [75]:
dfg_date_filtered.groupby(['date','location_name']).aggregate('sum')

value
date       location_name       
2022-02-26 Alabama           14
           Alaska             1
           Arizona           72
           Arkansas          42
           California        34
...                         ...
2022-10-15 Virginia          52
           Washington        17
           West Virginia     12
           Wisconsin          6
           Wyoming            3

[1537 rows x 1 columns]

In [76]:
dfg_date_filtered['date'].unique()

array(['2022-02-26T00:00:00.000000000', '2022-03-05T00:00:00.000000000',
       '2022-03-12T00:00:00.000000000', '2022-03-19T00:00:00.000000000',
       '2022-03-26T00:00:00.000000000', '2022-04-02T00:00:00.000000000',
       '2022-04-09T00:00:00.000000000', '2022-04-16T00:00:00.000000000',
       '2022-04-23T00:00:00.000000000', '2022-04-30T00:00:00.000000000',
       '2022-05-07T00:00:00.000000000', '2022-05-14T00:00:00.000000000',
       '2022-05-21T00:00:00.000000000', '2022-05-28T00:00:00.000000000',
       '2022-06-04T00:00:00.000000000', '2022-06-11T00:00:00.000000000',
       '2022-07-23T00:00:00.000000000', '2022-07-30T00:00:00.000000000',
       '2022-08-06T00:00:00.000000000', '2022-08-13T00:00:00.000000000',
       '2022-08-20T00:00:00.000000000', '2022-08-27T00:00:00.000000000',
       '2022-09-03T00:00:00.000000000', '2022-09-10T00:00:00.000000000',
       '2022-09-17T00:00:00.000000000', '2022-09-24T00:00:00.000000000',
       '2022-10-01T00:00:00.000000000', '2022-10-08

#### Flu weekly prediction

In [77]:
# 2022-01-31-MOBS-GLEAM_FLUH.csv
# 2022-02-07-MOBS-GLEAM_FLUH.csv
# 2022-02-14-MOBS-GLEAM_FLUH.csv
# 2022-02-21-MOBS-GLEAM_FLUH.csv
# 2022-02-28-MOBS-GLEAM_FLUH.csv
# 2022-03-07-MOBS-GLEAM_FLUH.csv
# 2022-03-14-MOBS-GLEAM_FLUH.csv
# 2022-03-21-MOBS-GLEAM_FLUH.csv
# 2022-03-28-MOBS-GLEAM_FLUH.csv
# 2022-04-04-MOBS-GLEAM_FLUH.csv
# 2022-04-11-MOBS-GLEAM_FLUH.csv
# 2022-04-18-MOBS-GLEAM_FLUH.csv
# 2022-04-25-MOBS-GLEAM_FLUH.csv
# 2022-05-02-MOBS-GLEAM_FLUH.csv
# 2022-05-09-MOBS-GLEAM_FLUH.csv
# 2022-05-16-MOBS-GLEAM_FLUH.csv
# 2022-05-23-MOBS-GLEAM_FLUH.csv
# 2022-05-30-MOBS-GLEAM_FLUH.csv
# 2022-06-13-MOBS-GLEAM_FLUH.csv
# 2022-06-20-MOBS-GLEAM_FLUH.csv
# 2022-10-17-MOBS-GLEAM_FLUH.csv
# 2022-10-24-MOBS-GLEAM_FLUH.csv

In [78]:
# import required module
import os
# assign directory
directory = '/Users/xiangyikong/Desktop/week73/GLEAM_preds'
 
# iterate over files in
# that directory
files_name = []
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        files_name.append(f)
        # print(f)

In [79]:


weeks_1_df = pd.DataFrame()
weeks_2_df = pd.DataFrame()
weeks_3_df = pd.DataFrame()
weeks_4_df = pd.DataFrame()
for i in files_name:
    df = pd.read_csv(i)
    # if df['target'] == '1 wk ahead inc flu hosp':
    weeks_1_df = weeks_1_df.append(df[(df['target'] == '1 wk ahead inc flu hosp') & (df['type'] == 'point')])
    weeks_2_df = weeks_2_df.append(df[(df['target'] == '2 wk ahead inc flu hosp') & (df['type'] == 'point')])
    weeks_3_df = weeks_3_df.append(df[(df['target'] == '3 wk ahead inc flu hosp') & (df['type'] == 'point')])
    weeks_4_df = weeks_4_df.append(df[(df['target'] == '4 wk ahead inc flu hosp') & (df['type'] == 'point')])

weeks_1_df = weeks_1_df.drop(['target','type','quantile'],axis = 1)
weeks_2_df = weeks_2_df.drop(['target','type','quantile'],axis = 1)
weeks_3_df = weeks_3_df.drop(['target','type','quantile'],axis = 1)
weeks_4_df = weeks_4_df.drop(['target','type','quantile'],axis = 1)


weeks_1_df = weeks_1_df.sort_values(by=['target_end_date', 'location'])
weeks_2_df = weeks_2_df.sort_values(by=['target_end_date', 'location'])
weeks_3_df = weeks_3_df.sort_values(by=['target_end_date', 'location'])
weeks_4_df = weeks_4_df.sort_values(by=['target_end_date', 'location'])


# weeks to drop
lst_weeks_drop = ['2022-06-18', '2022-06-25','2022-07-02','2022-07-09','2022-07-16', '2022-10-22', '2022-10-29', '2022-11-05', '2022-11-12', "2022-11-19"]
# states to drop 
states_to_drop = ['11']

weeks_1_df = weeks_1_df[~weeks_1_df['target_end_date'].isin(lst_weeks_drop)]
weeks_2_df = weeks_2_df[~weeks_2_df['target_end_date'].isin(lst_weeks_drop)]
weeks_3_df = weeks_3_df[~weeks_3_df['target_end_date'].isin(lst_weeks_drop)]
weeks_4_df = weeks_4_df[~weeks_4_df['target_end_date'].isin(lst_weeks_drop)]

weeks_1_df = weeks_1_df[~weeks_1_df['location'].isin(states_to_drop)]
weeks_2_df = weeks_2_df[~weeks_2_df['location'].isin(states_to_drop)]
weeks_3_df = weeks_3_df[~weeks_3_df['location'].isin(states_to_drop)]
weeks_4_df = weeks_4_df[~weeks_4_df['location'].isin(states_to_drop)]


# 1 week ahead
week_1_df_2 = weeks_1_df.drop_duplicates(subset=['target_end_date'])
week_1_df_3 = weeks_1_df.drop_duplicates(subset=['location'])
list1_week1 = list(week_1_df_2['target_end_date'][3:])
list2_week1 = list(week_1_df_3['location'])

# 2 week ahead
week_2_df_2 = weeks_2_df.drop_duplicates(subset=['target_end_date'])
week_2_df_3 = weeks_2_df.drop_duplicates(subset=['location'])
list1_week2 = list(week_2_df_2['target_end_date'][2:-1])
list2_week2 = list(week_2_df_3['location'])

# 3 week ahead
week_3_df_2 = weeks_3_df.drop_duplicates(subset=['target_end_date'])
week_3_df_3 = weeks_3_df.drop_duplicates(subset=['location'])
list1_week3 = list(week_3_df_2['target_end_date'][1:-2])
list2_week3 = list(week_3_df_3['location'])


# 4 week ahead
week_4_df_2 = weeks_4_df.drop_duplicates(subset=['target_end_date'])
week_4_df_3 = weeks_4_df.drop_duplicates(subset=['location'])
list1_week4 = list(week_4_df_2['target_end_date'][:-3])
list2_week4 = list(week_4_df_3['location'])



time_length = len(list1_week4)
state_length = len(list2_week4)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

#### Flu Prediction Matrix

In [ ]:
pred_week1 = np.zeros([time_length,state_length])
no_value_1_week = {}
for id1,i in enumerate(list1_week1):
    df4 = weeks_1_df.loc[weeks_1_df['target_end_date'] == i]
    for id2,j in enumerate(list2_week1):
        df5 = df4.loc[df4['location'] == j]
        try:
            pred_week1[id1,id2] = df5['value']
        except:
            # print('error')
            if i not in no_value_1_week.keys():
                no_value_1_week[i] = [j]
            else:
                no_value_1_week[i].append(j)
print(pred_week1.shape)

(16, 51)


In [ ]:
pred_week2 = np.zeros([time_length,state_length])
no_value_2_week = {}
for id1,i in enumerate(list1_week2):
    df4 = weeks_2_df.loc[weeks_2_df['target_end_date'] == i]
    for id2,j in enumerate(list2_week2):
        df5 = df4.loc[df4['location'] == j]
        try:
            pred_week2[id1,id2] = df5['value']
        except:
            # print('error')
            if i not in no_value_2_week.keys():
                no_value_2_week[i] = [j]
            else:
                no_value_2_week[i].append(j)
print(pred_week2.shape)

(16, 51)


In [ ]:
pred_week3 = np.zeros([time_length,state_length])
no_value_3_week = {}
for id1,i in enumerate(list1_week3):
    df4 = weeks_3_df.loc[weeks_3_df['target_end_date'] == i]
    for id2,j in enumerate(list2_week3):
        df5 = df4.loc[df4['location'] == j]
        try:
            pred_week3[id1,id2] = df5['value']
        except:
            # print('error')
            if i not in no_value_3_week.keys():
                no_value_3_week[i] = [j]
            else:
                no_value_3_week[i].append(j)
print(pred_week3.shape)

(16, 51)


In [ ]:
pred_week4 = np.zeros([time_length,state_length])
no_value_4_week = {}
for id1,i in enumerate(list1_week4):
    df4 = weeks_4_df.loc[weeks_4_df['target_end_date'] == i]
    for id2,j in enumerate(list2_week4):
        df5 = df4.loc[df4['location'] == j]
        try:
            pred_week4[id1,id2] = df5['value']
        except:
            # print('error')
            if i not in no_value_4_week.keys():
                no_value_4_week[i] = [j]
            else:
                no_value_4_week[i].append(j)
print(pred_week4.shape)

(16, 51)


In [ ]:
flu_pred = np.stack([pred_week1,pred_week2,pred_week3,pred_week4],axis=0)
flu_pred.shape

(4, 16, 51)

#### Residuals

In [ ]:
flu_residual = flu_pred - flu_groundtruth
residual_trans = flu_residual.transpose(1,2,0)
print(residual_trans.shape)

(16, 51, 4)


In [ ]:
#x data

x_num_samples, x_num_nodes = 16, 51
x_data = residual_trans
# x_data = residual.transpose(1,2,0)

x_day = list(range(0,x_num_samples))

x_day = np.tile(x_day, [1, x_num_nodes, 1]).transpose((2, 1, 0))
x_data = np.concatenate([x_data,x_day], axis=-1)
print(x_data[:,:,4])

[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.
   2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.
   2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.]
 [ 3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.
   3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.
   3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.  3.]
 [ 4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.
   4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.
   4.  4

In [ ]:
x_data.shape

(16, 51, 5)

In [106]:
x1 = np.stack(x_data, axis=0)[:12]

In [107]:
x1.shape

(12, 51, 5)

In [110]:
x_test = np.stack(x_data, axis=0)[15:]

In [111]:
x_test.shape

(1, 51, 5)

In [ ]:
# x1 and x test 

In [89]:
residual_trans.shape

(16, 51, 4)

In [97]:
y_data = np.stack([residual_trans[1:13,:,0],residual_trans[2:14,:,1],
                        residual_trans[3:15,:,2],residual_trans[4:16,:,3]],axis=-1)

In [92]:
residual_trans[1:2,:,0].shape

(1, 51)

In [102]:
y1 = np.expand_dims(y_data,axis=1)

In [103]:
y1 = y1.transpose(0,3,2,1)

In [104]:
y1.shape

(12, 4, 51, 1)

In [105]:
y_test = np.zeros([1,4,51,1])

In [112]:
x_shuffle, y_shuffle = shuffle_in_unison(x1, y1)

num_samples = x1.shape[0]
num_val = int(num_samples*0.5)
num_train = num_samples-num_val
# train
x_train, y_train = x_shuffle[:num_train], y_shuffle[:num_train]

# val
x_val, y_val = (
    x_shuffle[num_train: num_train + num_val],
    y_shuffle[num_train: num_train + num_val],
)

In [113]:
for cat in ["train", "val", "test"]:
    _x, _y = locals()["x_" + cat], locals()["y_" + cat]
    print(cat, "x: ", _x.shape, "y:", _y.shape)
    np.savez_compressed(
        os.path.join("week73/", "%s.npz" % cat),
        x=_x,
        y=_y
        # x_offsets=x_offsets.reshape(list(x_offsets.shape) + [1]),
        # y_offsets=y_offsets.reshape(list(y_offsets.shape) + [1]),
    )

train x:  (6, 51, 5) y: (6, 4, 51, 1)
val x:  (6, 51, 5) y: (6, 4, 51, 1)
test x:  (1, 51, 5) y: (1, 4, 51, 1)


In [114]:
x1.shape

(12, 51, 5)

In [115]:
y1.shape

(12, 4, 51, 1)